In [1]:
!pip install langchain --quiet
!pip install transformers --quiet
!pip install accelerate --quiet
!pip install bitsandbytes --quiet
!pip install faiss-cpu --quiet
!pip install sentence_transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.4/648.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline,AutoModelForSeq2SeqLM

model_id = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map="auto")
pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512
)
# model = "gpt2-medium"
# tokenizer = AutoTokenizer.from_pretrained(model)
# model = AutoModelForCausalLM.from_pretrained(model)


# pipeline = pipeline(
#     "text-generation", 
#     model=model, 
#     tokenizer=tokenizer, 
#     max_length=100
# )

# model.half().cuda()

local_llm = HuggingFacePipeline(pipeline=pipeline)

# inputs = tokenizer("What's your mood today?", return_tensors="pt").to("cuda")
# tokens = model.generate(
#   **inputs,
#   max_new_tokens=64,
#   temperature=0.7,
#   do_sample=True,
# )

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --targ

In [4]:
from langchain import PromptTemplate, OpenAI, LLMChain
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

In [5]:

question = "What is the capital of England?"

print(llm_chain.run(question))

England is the capital of England. The capital of England is London. So, the answer is London.


In [6]:
import requests
import re
url = "https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt"
res = requests.get(url)
text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", res.text)

with open("state_of_the_union.txt","w+") as f :
  f.write(text)

In [7]:
from langchain.document_loaders import TextLoader
loader = TextLoader('/content/state_of_the_union.txt')
documents = loader.load()

In [8]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [9]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(local_llm, chain_type="stuff")

In [10]:
from langchain.vectorstores import FAISS
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()
db = FAISS.from_documents(docs, embeddings)

In [11]:
query = "What did the president say about the Supreme Court"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

Token indices sequence length is longer than the specified maximum sequence length for this model (7478 > 512). Running this sequence through the model will result in indexing errors


'The Supreme Court is the only nation on Earth that has always turned every crisis we have faced into an opportunity'

In [14]:
query = "What is the blackhole?"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

'The black hole'

In [15]:
print(str(docs[0].page_content))

Madam Speaker Madam Vice President our First Lady and Second Gentleman Members of Congress and the Cabinet Justices of the Supreme Court My fellow Americans  Last year COVID19 kept us apart This year we are finally together again Tonight we meet as Democrats Republicans and Independents But most importantly as Americans With a duty to one another to the American people to the Constitution And with an unwavering resolve that freedom will always triumph over tyranny Six days ago Russias Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways But he badly miscalculated He thought he could roll into Ukraine and the world would roll over Instead he met a wall of strength he never imagined He met the Ukrainian people From President Zelenskyy to every Ukrainian their fearlessness their courage their determination inspires the world Groups of citizens blocking tanks with their bodies Everyone from students to retirees teachers turned

In [18]:
docs[0].__dict__["metadata"]

{'source': '/content/state_of_the_union.txt'}